<a href="https://colab.research.google.com/github/rodmello/OracleTAMTools/blob/main/AZUL_Ticket_Cat_Learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer

# Replace 'your_file.xlsx' with the actual file name
# Replace 'Sheet1' with the actual sheet name
file_path = '/content/sample_data/AZUL_03DEC2024.xlsx'
sheet_name = 'CONTROLE_AZUL'

try:
    # Read the Excel file into a pandas DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name)
except FileNotFoundError:
    print(f"Error: File '{file_path}' not found.")
    exit()
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    exit()

# Define input and output columns
input_cols = ['Descrição', 'Módulo', 'Notas de fechamento']
output_cols = ['Class_Resolução_L1', 'Class_Resolução_L2', 'Módulo2', 'Processo']

# Filter out rows with all input values missing
df = df[df[input_cols].notna().any(axis=1)]

# Impute missing values separately for numerical and categorical features
for col in input_cols:
    if df[col].dtype == 'object':  # Impute categorical features with most frequent
        # Convert all values to strings before imputation to avoid mixed data types
        df[col] = df[col].astype(str)
        imputer = SimpleImputer(strategy='most_frequent')
        df[col] = imputer.fit_transform(df[[col]])[:, 0]  # Extract the imputed values from the column
    else:  # Impute numerical features with mean
        imputer = SimpleImputer(strategy='mean')  # or 'median' for numerical features
        df[col] = imputer.fit_transform(df[[col]])[:, 0]  # Extract the imputed values from the column

# Convert text data into numerical representations (one-hot encoding)
X = pd.get_dummies(df[input_cols])
y = pd.get_dummies(df[output_cols])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForestClassifier (you can experiment with other classifiers)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

# Example prediction for new data
new_data = pd.DataFrame({'Descrição': ['value1'], 'Módulo': ['value2'], 'Notas de fechamento': ['value3']}) # Use the correct input column names
new_data_encoded = pd.get_dummies(new_data)

# Align columns between new_data_encoded and X_train
missing_cols = set(X_train.columns) - set(new_data_encoded.columns)
for c in missing_cols:
    new_data_encoded[c] = 0
new_data_encoded = new_data_encoded[X_train.columns]

predicted_probabilities = model.predict_proba(new_data_encoded)

# Get predicted classes (you might want to adjust the threshold)
predicted_classes = model.predict(new_data_encoded)

# Convert numerical predictions back into text labels
predicted_labels = pd.DataFrame(predicted_classes, columns=y.columns) # Use y.columns to get the correct output column names
# Get the original labels from y
original_labels = y.columns.to_frame(index=False, name='original_labels')
original_labels['encoded_labels'] = original_labels.index
label_mapping = original_labels.set_index('encoded_labels')['original_labels'].to_dict()

# Apply the label mapping to the predicted classes
# Instead of using argmax, directly map True values to their corresponding labels
predicted_labels = predicted_labels.applymap(lambda x: label_mapping[x.argmax()] if isinstance(x, (np.ndarray, list)) else label_mapping[x] if x else None)

print("Predicted labels:")
display(predicted_labels)

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_divisi

              precision    recall  f1-score   support

           0       0.75      0.43      0.55        28
           1       0.00      0.00      0.00        19
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         0
          17       0.00    

<ipython-input-25-30c3dbedb22d>:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_data_encoded[c] = 0
<ipython-input-25-30c3dbedb22d>:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_data_encoded[c] = 0
<ipython-input-25-30c3dbedb22d>:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_dat

,Class_Resolução_L1_Análise/ How To,Class_Resolução_L1_Incidente,Class_Resolução_L1_Manutenção,Class_Resolução_L2_Acessos,Class_Resolução_L2_Acompanhamento Execução Integração,Class_Resolução_L2_Acompanhamento Execução Processo/ Transação,Class_Resolução_L2_Acompanhamento Execução Relatório,Class_Resolução_L2_Ajuste Relatório Custom,Class_Resolução_L2_Ajuste manual GL,Class_Resolução_L2_Análise de Dados/ Resultados de Relatório Custom,...,Processo_Restituição,Processo_Reversão Lotes,Processo_SOC Reports,Processo_Smart View,Processo_Solicitação Reembolso,Processo_Todos,Processo_Transação Externa,Processo_User Interface,Processo_Usuários,Processo_Validação NF
0,Class_Resolução_L1_Incidente,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
